In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [0]:
# load data
data = pd.read_csv(
    "https://raw.githubusercontent.com/Lambda-Spotify-Song-Suggester-3/datascience/master/kaggle_data/encoded.csv")
df = data.copy()

dictionary = df[['artist_name', 'track_name', 'track_key', 'track_id']]

In [108]:
print(df.shape)
df.head()

(130663, 19)


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity,track_key,artist_key
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15,1,1
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0,2,1
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56,3,2
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0,4,3
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0,5,3


In [109]:
dictionary.head()

,artist_name,track_name,track_key,track_id
0,YG,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",1,2RM4jf1Xa9zPgMGRDiht8O
1,YG,BAND DRUM (feat. A$AP Rocky),2,1tHDG53xJNGsItRA3vfVgs
2,R3HAB,Radio Silence,3,6Wosx2euFPMT14UXiWudMy
3,Chris Cooq,Lactose,4,3J2Jpw61sO7l6Hc7qdYV91
4,Chris Cooq,Same - Original mix,5,2jbYvQCyPgX3CdmAzeVeuS


In [0]:
# drop columns for training
df = df.drop(
    columns=[
        'artist_name',
        'track_id',
        'track_name'])


In [111]:
df.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity,track_key,artist_key
0,0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15,1,1
1,0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0,2,1
2,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56,3,2
3,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0,4,3
4,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0,5,3


In [0]:
# Scale the data
scaler = StandardScaler()
df_s = scaler.fit_transform(df)

In [112]:
df_s

array([[-0.97408012,  0.84982596,  0.20900459, ..., -0.46715031,
        -1.7321124 , -1.17912281],
       [-0.92032465,  1.39171405,  0.01759476, ..., -1.22806504,
        -1.73208584, -1.17912281],
       [-0.91858874,  0.11327905, -0.59859825, ...,  1.61268329,
        -1.73205928, -1.17902738],
       ...,
       [-0.03616588,  0.71303868, -0.59634092, ...,  1.15613445,
         1.72952995, -1.06823843],
       [-0.96806228,  1.00239639, -0.26487136, ...,  1.3083174 ,
         1.72955651,  0.46200407],
       [ 0.59744266,  0.70251658, -0.54245743, ...,  1.56195565,
         1.72958307, -1.03579378]])

In [113]:
df_s.shape

(130663, 16)

In [114]:
# X_train, X_test split
from sklearn.model_selection import train_test_split

X_train, X_test= train_test_split(df_s, test_size=0.33)
X_train.shape, X_test.shape

((87544, 16), (43119, 16))

In [115]:
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
print(X_train.shape)
print(X_test.shape)

(87544, 16)
(43119, 16)


In [0]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)
# X_train.shape, X_test.shape

In [0]:
# neural networks
import os
import datetime
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import TensorBoard
# this is the size of our encoded representations
encoding_dim = 1000

# this is our input placeholder
input_shape = Input(shape=(16,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation="relu")(input_shape)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(16, activation="softmax")(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_shape, decoded)

In [0]:
autoencoder.compile(optimizer='nadam', loss='binary_crossentropy')

In [118]:

stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir)

autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=64,
                shuffle=True,
                validation_data=(X_test, X_test),
                verbose = 1,
                callbacks=[stop, tensorboard])

Epoch 1/100
1368/1368 [==============================] - 4s 3ms/step - loss: -4.8700 - val_loss: -5.3069
Epoch 2/100
1368/1368 [==============================] - 4s 3ms/step - loss: -5.3511 - val_loss: -5.3725
Epoch 3/100
1368/1368 [==============================] - 5s 3ms/step - loss: -5.3776 - val_loss: -5.3822
Epoch 4/100
1368/1368 [==============================] - 5s 3ms/step - loss: -5.3841 - val_loss: -5.3861
Epoch 5/100
1368/1368 [==============================] - 4s 3ms/step - loss: -5.3867 - val_loss: -5.3886
Epoch 6/100
1368/1368 [==============================] - 4s 3ms/step - loss: -5.3883 - val_loss: -5.3896
Epoch 7/100
1368/1368 [==============================] - 4s 3ms/step - loss: -5.3893 - val_loss: -5.3904
Epoch 8/100
1368/1368 [==============================] - 4s 3ms/step - loss: -5.3901 - val_loss: -5.3909
Epoch 9/100
1368/1368 [==============================] - 4s 3ms/step - loss: -5.3907 - val_loss: -5.3915
Epoch 10/100
1368/1368 [==============================]

In [0]:
input_shape = Input(shape=(16,))
encoded = Dense(10, activation='elu')(input_shape)
encoded = Dense(5, activation='elu')(encoded)

encoded = Dense(3, activation='elu')(encoded) #=> This is the dry fruit

decoded = Dense(5, activation='elu')(encoded)
decoded = Dense(10, activation='elu')(decoded)
decoded = Dense(16, activation="softmax")(decoded)

In [120]:
# compile & fit model

autoencoder = Model(input_shape, decoded)

autoencoder.compile(optimizer='adadelta',
                    loss='binary_crossentropy')

stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=2)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir=logdir)

model = autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_data=(X_test, X_test),
                verbose = 1,
                callbacks=[stop, tensorboard])

Epoch 1/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0633 - val_loss: 0.0608
Epoch 2/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0572 - val_loss: 0.0547
Epoch 3/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0511 - val_loss: 0.0485
Epoch 4/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0449 - val_loss: 0.0422
Epoch 5/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0385 - val_loss: 0.0358
Epoch 6/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0320 - val_loss: 0.0290
Epoch 7/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0251 - val_loss: 0.0220
Epoch 8/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0179 - val_loss: 0.0145
Epoch 9/100
2736/2736 [==============================] - 6s 2ms/step - loss: 0.0103 - val_loss: 0.0068
Epoch 10/100
2736/2736 [==============================] - 6s 2ms/step - l

In [0]:
# pickle model
import pickle
# Its important to use binary mode 
Pickle = open('pickle_file', 'wb') 

# source, destination 
pickle.dump(model, Pickle)                      


# load the model from disk
loaded_model = pickle.load(open('pickle_file', 'rb'))

In [0]:
# def predictor(track_key):
#     '''
#     Function to take "track key" of a song of interest from dataframe, and
#     return a list of track_ids of the closest related 10songs.

#     output format:
#   ['4fbaKWFRghusXd4bSBvvfN']

#     '''

#     # Convert "track_key" to the index of the song
#     input_dictionary_entry = dictionary[dictionary['track_key'] == track_key]
#     input_index = input_dictionary_entry.index[0]

#     # Nearest Neighbors model
#     nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
#     nn.fit(df_s)

#     neighbor_predictions = nn.kneighbors([df_s[input_index]])

#     # This is a list of the INDEXES of the songs
#     list_of_predictions = neighbor_predictions[1][0].tolist()

#     similar_tracks = []
#     for item in list_of_predictions:
#         track_hash = dictionary['track_id'].iloc[item]
#         similar_tracks.append(track_hash)

#     return similar_tracks

In [0]:
# # testing functionality
# track_key = 1
# print(predictor(track_key))

['2RM4jf1Xa9zPgMGRDiht8O', '02s1Voowwhr0qTSOrMVEXk', '6PIdcDdMuoyAWeVqMRMLlR', '0MiAP4EQGIuikH23RTyP53', '2lD4dQpcac8H2Rq5nttJJh', '07lPxulz2gpCAVvgemzedl', '5MccaBOgJh4MsJNoj72eij', '3QBTZEqaCbJxztWh5NTKHs', '6yoM6KoDpWXKIWo9UnZ3Ef', '7Jbt6KWzS8cpP1xnW0tQ96']


In [0]:
# # song features, for plotting
# def feature_average(track_key):
#     '''
#     This function returns the sum of the features for the ten recommended songs.
#     '''
#     similar_tracks = predictor(track_key)
#     # Return a dataframe with only the ten most similar tracks
#     similar_tracks = data[data["track_id"].isin(similar_tracks)]
#     similar_tracks = similar_tracks[['acousticness', 'danceability',
#                                      'energy', 'instrumentalness',
#                                      'liveness', 'mode',
#                                      'speechiness', 'valence']]
#     # Average features of ten tracks
#     acousticness = round(similar_tracks['acousticness'].mean(), 2)
#     danceability = round(similar_tracks['danceability'].mean(), 2)
#     energy = round(similar_tracks['energy'].mean(), 2)
#     instrumentalness = round(similar_tracks['instrumentalness'].mean(), 2)
#     liveness = round(similar_tracks['liveness'].mean(), 2)
#     mode = round(similar_tracks['mode'].mean(), 2)
#     speechiness = round(similar_tracks['speechiness'].mean(), 2)
#     valence = round(similar_tracks['valence'].mean(), 2)
#     # Store all to "features" variable
#     features = []
#     attributes = [
#         acousticness,
#         danceability,
#         energy,
#         instrumentalness,
#         liveness,
#         mode,
#         speechiness,
#         valence]
#     # features.append(acousticness)
#     for attribute in attributes:
#         features.append(attribute)
#     return features

In [0]:
# # testing functionality
# print(feature_average(track_key))

[0.06, 0.73, 0.42, 0.0, 0.12, 1.0, 0.41, 0.25]


In [0]:
# type(feature_average(track_key))

list

In [0]:
# plotply radar-chart (https://plotly.com/python/radar-chart/)
import plotly.express as px
import pandas as pd

r =feature_average(track_key)

attributes = [
        'acousticness',
        'danceability',
        'energy',
        'instrumentalness',
        'liveness',
        'mode',
        'speechiness',
        'valence']

fig = px.line_polar(r=r, theta=attributes, line_close=True)
fig.update_traces(fill='toself')
fig.show()